In [5]:
# Package Imports
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv

load_dotenv()


True

In [6]:
# Create Variable for Odds APi Key
odds_api_key = os.getenv("ODDS_API")

type(odds_api_key)

str

In [8]:
# This response variable returns all current active sports and their related keys.
# Use if you need to see all potential sports to pick odds from or if you need to find your sports key.
# Does not cost API usage quota.

# Create the response variable
sports_response = requests.get(f"https://api.the-odds-api.com/v4/sports/?apiKey={odds_api_key}").json()

# Print the results
print(json.dumps(sports_response, indent=4, sort_keys=True))

''' "active": true,
        "description": "US Football",
        "group": "American Football",
        "has_outrights": false,
        "key": "americanfootball_nfl",
        "title": "NFL"'''

[
    {
        "active": true,
        "description": "US Football",
        "group": "American Football",
        "has_outrights": false,
        "key": "americanfootball_nfl",
        "title": "NFL"
    },
    {
        "active": true,
        "description": "Super Bowl Winner 2021/2022",
        "group": "American Football",
        "has_outrights": true,
        "key": "americanfootball_nfl_super_bowl_winner",
        "title": "NFL Super Bowl Winner"
    },
    {
        "active": true,
        "description": "Aussie Football",
        "group": "Aussie Rules",
        "has_outrights": false,
        "key": "aussierules_afl",
        "title": "AFL"
    },
    {
        "active": true,
        "description": "World Series Winner 2022",
        "group": "Baseball",
        "has_outrights": true,
        "key": "baseball_mlb_world_series_winner",
        "title": "MLB World Series Winner"
    },
    {
        "active": true,
        "description": "US Basketball",
        "group": "Ba

' "active": true,\n        "description": "US Football",\n        "group": "American Football",\n        "has_outrights": false,\n        "key": "americanfootball_nfl",\n        "title": "NFL"'

In [9]:
# Define variables for pull requests
SPORT = 'americanfootball_nfl' # use the sport_key from the /sports endpoint, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'american' # decimal | american

DATE_FORMAT = 'iso' # iso | unix

In [10]:
# This cell will process the API Call and create an odds_response variable to be printed below.
# This does count towards the API usage count.
odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds', params={
    'api_key': odds_api_key,
    'regions': REGIONS,
    'markets': MARKETS,
    'oddsFormat': ODDS_FORMAT,
    'dateFormat': DATE_FORMAT,
})

In [11]:
# This cell prints the API call above.

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(json.dumps(odds_json, indent=4, sort_keys=True))

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

Number of events: 4
[
    {
        "away_team": "Cincinnati Bengals",
        "bookmakers": [
            {
                "key": "unibet",
                "last_update": "2022-01-21T23:44:01Z",
                "markets": [
                    {
                        "key": "h2h",
                        "outcomes": [
                            {
                                "name": "Cincinnati Bengals",
                                "price": 158
                            },
                            {
                                "name": "Tennessee Titans",
                                "price": -179
                            }
                        ]
                    }
                ],
                "title": "Unibet"
            },
            {
                "key": "betrivers",
                "last_update": "2022-01-21T23:44:03Z",
                "markets": [
                    {
                        "key": "h2h",
                        "outcome

In [12]:
# Function to pull the weekly odds.
def weekly_odds(dump):
    odds_df = pd.DataFrame()
    for i in range(len(dump)):
        t1 = pd.DataFrame(dump[i]['bookmakers'][1]['markets'][0]['outcomes'][0], index=[0])
        t1 = t1.rename(columns={'name':'t1_name','price':'t1_odds'})
        t2 = pd.DataFrame(dump[i]['bookmakers'][1]['markets'][0]['outcomes'][1], index=[0])
        t2 = t2.rename(columns={'name':'t2_name','price':'t2_odds'}) 
        t1['away_team'] = dump[i]['away_team']
        t1['home_team'] = dump[i]['home_team']
        joined_odds = pd.concat([t1, t2], axis='columns')
        odds_df = odds_df.append(joined_odds)
         
    return odds_df

In [20]:
upcoming_games = weekly_odds(odds_json)
upcoming_games = upcoming_games.reset_index().drop(columns='index')
upcoming_games

,t1_name,t1_odds,away_team,home_team,t2_name,t2_odds
0,Cincinnati Bengals,158,Cincinnati Bengals,Tennessee Titans,Tennessee Titans,-180
1,Green Bay Packers,-235,San Francisco 49ers,Green Bay Packers,San Francisco 49ers,190
2,Los Angeles Rams,129,Los Angeles Rams,Tampa Bay Buccaneers,Tampa Bay Buccaneers,-152
3,Buffalo Bills,110,Buffalo Bills,Kansas City Chiefs,Kansas City Chiefs,-132


In [21]:
first_column = upcoming_games.pop('away_team')
second_column = upcoming_games.pop('home_team')
upcoming_games.insert(0,'away_team',first_column)
upcoming_games.insert(1,'home_team',second_column)

upcoming_games

,away_team,home_team,t1_name,t1_odds,t2_name,t2_odds
0,Cincinnati Bengals,Tennessee Titans,Cincinnati Bengals,158,Tennessee Titans,-180
1,San Francisco 49ers,Green Bay Packers,Green Bay Packers,-235,San Francisco 49ers,190
2,Los Angeles Rams,Tampa Bay Buccaneers,Los Angeles Rams,129,Tampa Bay Buccaneers,-152
3,Buffalo Bills,Kansas City Chiefs,Buffalo Bills,110,Kansas City Chiefs,-132


In [23]:
upcoming_games.to_csv("Resources/upcoming_games.csv")

    

In [28]:
Team_1 = f"{upcoming_games.iloc[0,2]} : {upcoming_games.iloc[0,3]}"
Team_2 = f"{upcoming_games.iloc[0,4]} : {upcoming_games.iloc[0,5]}"
display(Team_1)
display(Team_2)

'Cincinnati Bengals : 158'

'Tennessee Titans : -180'